In [ ]:
### Required
#### 1. **Use Binance Python SDK** to get public data


## Write a script (.py file)

## (a) get candle data(aka kline, histo)
## (b) get transactions(aka trades)
## (c) get market depth(aka orderbook)

#Format the data the way you like using pandas, save them seperately as csv.



In [1]:
from binance.client import Client
import pandas as pd
from datetime import datetime


## ref:https://stackoverflow.com/questions/43318383/converting-unix-timestamp-length-13-string-to-readable-date-in-python
def convert_to_date(timestamp_with_ms):
    '''convert timestamp in milliseconds to date'''
    dt = datetime.fromtimestamp(timestamp_with_ms / 1000)
    formatted_time = dt.isoformat(sep=' ', timespec='milliseconds')
    return formatted_time

def save_to_csv(df, filename):
    '''save dataframe to csv file'''
    # write dataframe to a csv with header without index
    df.to_csv(filename+'.csv', header=True, index=False)


In [2]:
##(a) get historical kline data from any date range
client = Client("", "")

print("Fetch 12hour klines for ETHBTC from 31 Dec, 2019 to 1 May, 2020")
# return: list of OHLCV values (Open, High, Low, Close, Volume) return: list of OHLCV values
klines = client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_12HOUR, "31 Dec, 2019", "1 May, 2020")


##load data to dataframe
df = pd.DataFrame(klines,columns=["Open time","Open","High", "Low","Close","Volume","Close time",\
                                "Quote asset volume","Number of trades","Taker buy base asset volume",
                                "Taker buy quote asset volume","Ignore"])
#drop column
df = df.drop(columns=["Ignore"])

# conver timstamp to date
df['Open time'] = df['Open time'].apply(lambda x: convert_to_date(x))
df["Close time"] = df["Close time"].apply(lambda x: convert_to_date(x))

# save to csv
filename = 'klines'
save_to_csv(df, filename)

Fetch 12hour klines for ETHBTC from 31 Dec, 2019 to 1 May, 2020


In [8]:
df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,2019-12-30 19:00:00.000,0.01816100,0.01830300,0.01812000,0.01821400,38596.53700000,2019-12-31 06:59:59.999,702.40052805,40246,20176.51500000,367.22789911
1,2019-12-31 07:00:00.000,0.01821600,0.01826200,0.01786700,0.01795300,59552.84900000,2019-12-31 18:59:59.999,1074.20233905,56942,27687.89200000,499.55414639
2,2019-12-31 19:00:00.000,0.01795400,0.01823600,0.01792000,0.01816500,30629.79000000,2020-01-01 06:59:59.999,553.49099402,39354,15537.70900000,280.79326543
3,2020-01-01 07:00:00.000,0.01816500,0.01835600,0.01805000,0.01815000,38487.77800000,2020-01-01 18:59:59.999,701.06497098,44750,19199.88800000,349.81774547
4,2020-01-01 19:00:00.000,0.01815200,0.01820900,0.01801000,0.01813900,30391.28300000,2020-01-02 06:59:59.999,550.77515963,36432,14362.62000000,260.31200234


In [3]:
## (b) get transactions(aka trades)

print("Get recent trades for BNBBTC")
trades = client.get_recent_trades(symbol='BNBBTC', limit=500)
df_trades = pd.DataFrame(trades)
df_trades["time"] = df_trades["time"].apply(lambda x: convert_to_date(x))

# save to csv
filename = 'transactions'
save_to_csv(df_trades, filename)

Get recent trades for BNBBTC


In [9]:
df_trades.head()

,id,price,qty,quoteQty,time,isBuyerMaker,isBestMatch
0,91527867,0.00251430,1.58000000,0.00397259,2020-10-11 13:39:28.888,False,True
1,91527868,0.00251450,1.00000000,0.00251450,2020-10-11 13:39:49.393,False,True
2,91527869,0.00251430,1.01000000,0.00253944,2020-10-11 13:39:50.136,True,True
3,91527870,0.00251430,5.00000000,0.01257150,2020-10-11 13:39:54.754,True,True
4,91527871,0.00251430,4.94000000,0.01242064,2020-10-11 13:39:54.773,True,True


In [4]:
## (c) get market depth
# bids:buyers, asks:sellers

print("Get order book for BNBBTC")
depth = client.get_order_book(symbol='BNBBTC', limit=100)

# formating data for dataframe
format_ls = ['bids',"asks"]
for i in format_ls:
    for idx, e in enumerate(depth[i]):
        e.append(i)

# build data frame
df1 = pd.DataFrame(depth["bids"], columns=["Price", "QTY", "Bids or Asks"])
df2 = pd.DataFrame(depth["asks"], columns=["Price", "QTY", "Bids or Asks"])
df_order_book = df1.append(df2)


# save to csv
filename='order_book'
save_to_csv(df_order_book, filename)

Get order book for BNBBTC


In [10]:
df_order_book.head()

,Price,QTY,Bids or Asks
0,0.00251540,275.31000000,bids
1,0.00251470,289.77000000,bids
2,0.00251460,0.36000000,bids
3,0.00251440,1.92000000,bids
4,0.00251370,278.00000000,bids


In [11]:
df_order_book.tail()

,Price,QTY,Bids or Asks
95,0.00252520,13.13000000,asks
96,0.00252530,13.30000000,asks
97,0.00252540,0.25000000,asks
98,0.00252550,36.95000000,asks
99,0.00252560,1.99000000,asks


In [5]:
#### Optional

#### 1. Trade in Binance by its python SDK

#- [ ] setup a binance account
#- [ ] setup its trading api(you need to create a api key, api secret pair to trade)
#- [ ] check python SDK, create a test order.

from binance.client import Client

# Created at Binance Not Binance.us. 
# They are different, so your frist attempt using the key and secret from Binance.us don't work
api_key='FbMo8RNCKlQl2yHsZ8nFWLeT2apXEZPagRZhIs3PoJrePErGQtBgIcGsKfB2q3Ld'
api_secret='YXMWxgfNYy1sJN6ofoA03LgX5aChY8BEeGsar3VxfjdmvN6SOAPrLKZvU0ryXz9o'

## Check the length of key and secret are correct
# print(len(api_key),len(api_secret)), output 64 64

client = Client(api_key, api_secret)
## check if client works
# client.get_account()

# place a test market buy order
order = client.create_test_order(
    symbol='BNBBTC',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=100)